<a href="https://colab.research.google.com/github/D3TaLES/databases_demo/blob/main/notebooks/processing_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Manually process absorption data

In [ ]:
# Import Required code 
import pandas as pd # import Pandas for file parsing
from scipy.signal import find_peaks # import SciPY for peak finding 

In [ ]:
# Pull example data files from GitHub
! rm -r databases_demo/ # remove database_demo directory if it already exists
! git clone https://github.com/D3TaLES/databases_demo.git # get code from GitHub repository 

rm: cannot remove 'databases_demo/': No such file or directory
Cloning into 'databases_demo'...
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 329 (delta 8), reused 19 (delta 6), pack-reused 304
Receiving objects: 100% (329/329), 681.98 KiB | 3.73 MiB/s, done.
Resolving deltas: 100% (176/176), done.


In [ ]:
# Use pandas package to read csv file
data = pd.read_csv("databases_demo/raw_data/uvvis_biphenyl.csv", header=3, names=['wavelength', 'absorbance'])
# Use scipy package to find the absorbance peaks
peaks, _ = find_peaks(data.absorbance, height=0.3)
# Find the wavelength values the correspond to the absorbance peaks
peaks_wavelength = [data.wavelength[p]for p in peaks]
# Find the first (lowest) wavelength value for the absorbance peaks
first_peak =  min(peaks_wavelength)

In [ ]:
# Calculate optical gap
optical_gap = round(1240 / first_peak, 3)
print(optical_gap)


3.69


In [ ]:
# Get final absorbance data to insert into database
absorbance_data = data.to_dict('records')
print(absorbance_data[:10]) # Show first 10 absorbance data records

[{'wavelength': 1005.0, 'absorbance': 0.000504422}, {'wavelength': 997.0, 'absorbance': 0.000583079}, {'wavelength': 989.0, 'absorbance': 0.000667289}, {'wavelength': 981.0, 'absorbance': 0.000756056}, {'wavelength': 973.0, 'absorbance': 0.000848101}, {'wavelength': 966.0, 'absorbance': 0.000941877}, {'wavelength': 958.0, 'absorbance': 0.001035605}, {'wavelength': 951.0, 'absorbance': 0.001127321}, {'wavelength': 944.0, 'absorbance': 0.001214939}, {'wavelength': 937.0, 'absorbance': 0.001296327}]
